###21720893 손명균 
팀은 없습니다.

In [6]:
import numpy as np
from datetime import datetime      # datetime.now() 를 이용하여 학습 경과 시간 측정

def sigmoid(x):         # sigmoid 함수
    return 1 / (1+np.exp(-x))

def cross_entropy(t, y) :
    delta = 1e-7    # log 무한대 발산 방지
    return -np.sum(t*np.log(y+delta) + (1-t)*np.log((1-y)+delta))    

def numerical_derivative(f, x):      # 수치미분 함수
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
    return grad

class LogicGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name
        # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(16,4)  # 16개의 입력데이터 x1 ~ x4 에 대하여 batch 처리 행렬
        self.tdata = tdata.reshape(16,2)  # 16개의 입력데이터 x1 ~ x4 에 대한 각각의 계산 값 행렬
        
        #  hidden layer unit : 5, 4, 5개 가정,  가중치 W2, 바이어스 b2 초기화
        self.W2 = np.random.rand(4,5)  # weight, 4 X 5 matrix
        self.b2 = np.random.rand(5)
        self.W3 = np.random.rand(5,4)
        self.b3 = np.random.rand(4)
        self.W4 = np.random.rand(4,5)
        self.b4 = np.random.rand(5)
        # 3층 output layer unit : 1 개 , 가중치 W3, 바이어스 b3 초기화
        self.W5 = np.random.rand(5,2)
        self.b5 = np.random.rand(2)
                        
        # 학습률 learning rate 초기화
        self.lr = 1e-2
        print(self.name + " object is created")
            
    def feed_forward(self):        # errFunc()함수 대신 feed forward를 통하여 손실함수(cross-entropy) 값 계산
        z2 = np.dot(self.xdata, self.W2) + self.b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                            # 은닉층의 출력
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        z4 = np.dot(a3, self.W4) + self.b4
        a4 = sigmoid(z4)
        z5 = np.dot(a4, self.W5) + self.b5
        y = a5 = sigmoid(z5)                        # 출력층의 출력
        return cross_entropy(self.tdata, y)         # 출력의 손실값 리턴
    
    def errValue(self):             # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        return  self.feed_forward()
    
    def train(self):            # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
        f = lambda x : self.feed_forward()
        start = datetime.now()
        for step in range(40001):
            self.W2 -= self.lr * numerical_derivative(f, self.W2)
            self.b2 -= self.lr * numerical_derivative(f, self.b2)
            self.W3 -= self.lr * numerical_derivative(f, self.W3)
            self.b3 -= self.lr * numerical_derivative(f, self.b3)
            self.W4 -= self.lr * numerical_derivative(f, self.W4)
            self.b4 -= self.lr * numerical_derivative(f, self.b4)
            self.W5 -= self.lr * numerical_derivative(f, self.W5)
            self.b5 -= self.lr * numerical_derivative(f, self.b5)
            if (step % 4000 == 0):
                print("Step = {:<5d}\tError Val = {:.4f}".format(step, self.errValue()))
        print("Training time = ", datetime.now() - start)

    def predict(self, test):      # query, 즉 미래 값 예측 함수
        result = []
        z2 = np.dot(test, self.W2) + self.b2         # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                             # 은닉층의 출력
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        z4 = np.dot(a3, self.W4) + self.b4
        a4 = sigmoid(z4)
        z5 = np.dot(a4, self.W5) + self.b5
        y = a5 = sigmoid(z5)                        # 출력층의 출력
        if y[0] > 0.5:
          result.append(1)  # True
        else:
          result.append(0)  # False
        if y[1] > 0.5:
          result.append(1)
        else:
          result.append(0)
        return result, y

In [7]:
xdata = np.array([ [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1],
                  [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1],
                  [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1],
                  [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]])
tdata = []
for i in range(16):
  res1 = xdata[i][0]^xdata[i][1]^xdata[i][2]^xdata[i][3]
  res2 = (xdata[i][0]^xdata[i][1]^xdata[i][2]) or xdata[i][3]
  res = []
  res.append(res1)
  res.append(res2)
  tdata.append(res)
tdata = np.array(tdata)
#tdata
#tdata = np.array([[0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0],
#                  [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1]])

array([[0, 0],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 1]])

In [13]:
calc = LogicGate("Calc", xdata, tdata)
calc.train() 

test_data = np.array([ [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1],
                  [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1],
                  [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1],
                  [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1],
                  [0.2, 0.2, 0.1, 0.1], [0.9, 0.2, 0.1, 0.7], [0.7, 0.6, 0.1, 0.8]])
for data in test_data:
    r, y = calc.predict(data)
    print(data, "-->", r, "({:.3f},{:.3f})".format(y[0], y[1]))

Calc object is created
Step = 0    	Error Val = 35.0247
Step = 4000 	Error Val = 20.0823
Step = 8000 	Error Val = 20.0754
Step = 12000	Error Val = 20.0458
Step = 16000	Error Val = 16.7951
Step = 20000	Error Val = 16.6500
Step = 24000	Error Val = 16.6416
Step = 28000	Error Val = 16.6329
Step = 32000	Error Val = 8.6226
Step = 36000	Error Val = 0.3430
Step = 40000	Error Val = 0.1533
Training time =  0:09:51.563934
[0. 0. 0. 0.] --> [0, 0] (0.009,0.005)
[0. 0. 0. 1.] --> [1, 1] (0.992,0.998)
[0. 0. 1. 0.] --> [1, 1] (0.994,0.997)
[0. 0. 1. 1.] --> [0, 1] (0.005,1.000)
[0. 1. 0. 0.] --> [1, 1] (0.994,0.998)
[0. 1. 0. 1.] --> [0, 1] (0.008,1.000)
[0. 1. 1. 0.] --> [0, 0] (0.009,0.005)
[0. 1. 1. 1.] --> [1, 1] (0.993,0.998)
[1. 0. 0. 0.] --> [1, 1] (0.994,0.997)
[1. 0. 0. 1.] --> [0, 1] (0.005,1.000)
[1. 0. 1. 0.] --> [0, 0] (0.012,0.007)
[1. 0. 1. 1.] --> [1, 1] (0.993,0.998)
[1. 1. 0. 0.] --> [0, 0] (0.009,0.005)
[1. 1. 0. 1.] --> [1, 1] (0.993,0.998)
[1. 1. 1. 0.] --> [1, 1] (0.995,0.997)
